In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tensorflow-speech-recognition-challenge/link_to_gcp_credits_form.txt
/kaggle/input/tensorflow-speech-recognition-challenge/test.7z
/kaggle/input/tensorflow-speech-recognition-challenge/sample_submission.7z
/kaggle/input/tensorflow-speech-recognition-challenge/train.7z


In [25]:
import os
import re
import numpy as np
import pandas as pd
from glob import glob
from scipy import signal
from scipy.io import wavfile
from scipy.fftpack import fft
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [9]:
!apt-get install -y p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  p7zip
Suggested packages:
  p7zip-rar
The following NEW packages will be installed:
  p7zip p7zip-full
0 upgraded, 2 newly installed, 0 to remove and 23 not upgraded.
Need to get 1522 kB of archives.
After this operation, 5755 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 p7zip amd64 16.02+dfsg-6 [358 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 p7zip-full amd64 16.02+dfsg-6 [1164 kB]
Fetched 1522 kB in 3s (605 kB/s)      
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package p7zip.
(Reading database ... 95165 files and directories currently installed.)
Preparing to unpack .../p7zip_16.02+dfsg-6_amd64.deb ...
Unpacking p7zip (16.02+dfsg-6) ...
Selecting previously unselected package p7zip-full.
Preparin

In [10]:
!7z x ../input/tensorflow-speech-recognition-challenge/train.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ../input/tensorflow-speech-recognition-challenge                                                          1 file, 1121103842 bytes (1070 MiB)

Extracting archive: ../input/tensorflow-speech-recognition-challenge/train.7z
--
Path = ../input/tensorflow-speech-recognition-challenge/train.7z
Type = 7z
Physical Size = 1121103842
Headers Size = 389133
Method = Delta LZMA2:24
Solid = +
Blocks = 2

      0% 38 - train/audio/_background_noise_/doing_the_dishes.wa                                                              0% 39 - train/audio/_background_noise_/dude_miaowing.w                                                          0% 41 - train/audio/_background_noise_/pink_noise.wa                                                        0% 42 - train/audio/_ba

In [28]:
train_data = "train/audio"
labels = os.listdir("train/audio")
labels.sort()
print(labels)
test_labels = ["yes", "up", "no", "down", "left", "right", "on", "off", "stop", "go", "silence", "unknown"]
print(test_labels)
print(len(test_labels))

['_background_noise_', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']
['yes', 'up', 'no', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown']
12


In [29]:
def Audio_Padding(audio):
    if len(audio) >= sample_length:
        return audio
    else:
        return np.pad(audio, pad_width = (sample_length - len(audio), 0), mode='constant', constant_values=(0, 0))

def Audio_Chopping(audio):
    for i in range(20):
        temp = np.random.randint(0, len(audio) - sample_length)
        yield samples[temp: temp+sample_length]

In [30]:
def Log_spectrogram(audio, sample_rate, window_size=20, step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,fs=sample_rate,window='hann',nperseg=nperseg,noverlap=noverlap,detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

In [31]:
def List_sample_names(path):
    ext='wav'
    fpaths = glob(os.path.join(path, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

In [32]:
def Change_labels(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in legal_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))

In [38]:
labels, file_names = List_sample_names(train_data)
x_train = []
y_train = []
sample_length_new = 8000

for label, file in zip(labels, file_names):
    sample_rate, samples = wavfile.read(os.path.join(train_data, label, file))
    samples = Audio_Padding(samples)
    if len(samples) > 16000:
        n_samples = Audio_Chopping(samples)
    else: n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(sample_length_new / sample_length * samples.shape[0]))
        try:
            _, _, specgram = Log_spectrogram(resampled, sample_rate=sample_length_new)
        except:
            print(label, samples)
        y_train.append(label)
        x_train.append(specgram)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: WavFileWarning: Chunk (non-data) not understood, skipping it.
  import sys


ValueError: invalid number of data points (0) specified

In [45]:
labels, fnames = List_sample_names(train_data_path)

sample_length_new = 8000
y_train = []
x_train = []

for label, fname in zip(labels, fnames):
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
    samples = Audio_Padding(samples)
    if len(samples) > 16000:
        n_samples = Audio_Chopping(samples)
    else: n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(sample_length_new / sample_length * samples.shape[0]))
        _, _, specgram = Log_spectrogram(resampled, sample_rate=new_sample_rate)
        y_train.append(label)
        x_train.append(specgram)
x_train = np.array(x_train)
x_train = x_train.reshape(tuple(list(x_train.shape) + [1]))
y_train = label_transform(y_train)
label_index = y_train.columns.values
y_train = y_train.values
y_train = np.array(y_train)

train/audio


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: WavFileWarning: Chunk (non-data) not understood, skipping it.
  


In [47]:
print(x_train.shape)
print(y_train.shape)

(64841, 99, 81, 1)
(64841, 12)


In [51]:
input_shape = (99, 81, 1)
nclass = 12
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Conv2D(8, kernel_size=2, activation="relu")(norm_inp)
img_1 = Conv2D(8, kernel_size=2, activation="relu")(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Conv2D(16, kernel_size=3, activation="relu")(img_1)
img_1 = Conv2D(16, kernel_size=3, activation="relu")(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Conv2D(32, kernel_size=3, activation="relu")(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(Dense(128, activation="relu")(img_1))
dense_1 = BatchNormalization()(Dense(128, activation="relu")(dense_1))
dense_1 = Dense(nclass, activation="softmax")(dense_1)

model = Model(inputs=inp, outputs=dense_1)
opt = Adam()

model.compile(optimizer=opt, loss = "binary_crossentropy", metrics=['accuracy'])
model.summary()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=2017)
model.fit(x_train, y_train, batch_size=16, validation_data=(x_valid, y_valid), epochs=10, shuffle=True)

model.save('cnn.model')

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 99, 81, 1)]       0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 99, 81, 1)         4         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 98, 80, 8)         40        
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 97, 79, 8)         264       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 48, 39, 8)         0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 48, 39, 8)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 46, 37, 16)        1168

KeyboardInterrupt: 

In [52]:
model.save('new_model.model')

In [58]:
!7z x ../input/tensorflow-speech-recognition-challenge/test.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ../input/tensorflow-speech-recognition-challenge                                                          1 file, 2640679130 bytes (2519 MiB)

Extracting archive: ../input/tensorflow-speech-recognition-challenge/test.7z
--
Path = ../input/tensorflow-speech-recognition-challenge/test.7z
Type = 7z
Physical Size = 2640679130
Headers Size = 1345163
Method = Delta LZMA2:24
Solid = +
Blocks = 3

      0% 83 - test/audio/clip_00219963b.wa                                        0% 181 - test/audio/clip_00465ae81.w                                        0% 300 - test/audio/clip_007960eac.w                                        0% 378 - test/audio/clip_009fa6856.w                                        0% 492 - test/audio/clip_00da5ab6d.w                        

In [61]:
test_data_path = 'test/audio'
print(len(os.listdir(test_data_path)))

158538


In [70]:
def test_data_generator(batch=16):
    fpaths = glob(os.path.join(test_data_path, '*wav'))
    i = 0
    j=0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        rate, samples = wavfile.read(path)
        samples = pad_audio(samples)
        resampled = signal.resample(samples, int(new_sample_rate / rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        imgs.append(specgram)
        fnames.append(path.split('\\')[-1])
        if i == batch:
            
            i = 0
#             j+=1
#             print(j)
            imgs = np.array(imgs)
            imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)
        imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
        yield fnames, imgs
    #raise StopIteration()

In [71]:
index = []
results = []
for fnames, imgs in test_data_generator(batch=32):
    predicts = model.predict(imgs)
    predicts = np.argmax(predicts, axis=1)
    predicts = [label_index[p] for p in predicts]
    index.extend(fnames)
    results.extend(predicts)

df = pd.DataFrame(columns=['fname', 'label'])
df['fname'] = index
df['label'] = results
#df.to_csv(os.path.join(out_path, 'sub.csv'), index=False)

NameError: name 'out_path' is not defined

In [82]:
print(type(index))
index2 = []
for i in index:
    index2.append(i[11:])

<class 'list'>


In [84]:
print(len(index2))

df2 = pd.DataFrame(columns=['fname', 'label'])
df2['fname'] = index2
df2['label'] = results
df2.to_csv(os.path.join(r'.', 'file.csv'), index=False)

158538


In [85]:
file2 = pd.read_csv("file.csv")

In [73]:
df.to_csv(os.path.join(r'.', 'sub.csv'), index=False)

In [76]:
file = pd.read_csv('sub.csv')

In [78]:
print(type(file))
print(file.)

<class 'pandas.core.frame.DataFrame'>
158538
